In [1]:
# Assign directory
import os
BRANDON = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
YASH = '/Users/yashd/Desktop/hierarchical-bayesian-model-validation/'

ROOT_DIR = BRANDON
os.chdir(ROOT_DIR + 'testing-framework/')

DATA_NAME = 'toy-agriVision-red-wavelet'
GROUP = 'layer'
SKIP_OPTIMIZE_STEP = True

# os.mkdir(DATA_NAME)
# os.mkdir(os.path.join(DATA_NAME, "CSVs"))
# os.mkdir(os.path.join(DATA_NAME, "plots"))

In [2]:
from utilities import *
from plot_utilities import *
''' Comment out the below two lines if you do not have Matlab installed and opened'''
import matlab.engine 
eng = matlab.engine.connect_matlab()

In [3]:
def compute_prior_pdf(r, eta, n_samples = 10000, tail_bound = 0.05, tail_percent = 0.01, scale = 1, scipy_int=True, eng= None, debug=False):
    
    """
    Computes the prior probability density function (PDF) for given parameters r and eta.

    Inputs:
    - r (float): Shape parameter for the generalized gamma distribution, recommended r >= 0.4
    - eta (float): Parameter related to the beta in the generalized gamma distribution, recommended eta >= 0
    - n_samples (int, optional): Number of samples to generate. Default is 10000
    - tail_bound (float, optional): Bound for the tail of the distribution. Default is 0.05
    - tail_percent (float, optional): Percentage of samples to allocate to the tail. Default is 0.01
    - scale (float, optional): Scale parameter for the distribution. Default is 1
    - scipy_int (bool, optional): If True, uses SciPy for integration. If False, uses 'eng'. Default is True
    - eng (object, optional): MATLAB engine for computation if scipy_int is False. Default is None
    - debug (bool, optional): If True, prints debug information. Default is False

    Outputs:
    - xs (numpy.ndarray): Array of x values
    - pdf (numpy.ndarray): Corresponding PDF values

    Note: The function uses a combination of linear and logarithmic spacing for x values to capture both the central part and the tails of the distribution.
    With 10000 samples and relatively large eta values, computed CDFs are reliable for r >= 0.4. For more robust CDFs, use MATLAB. 
    Though the distribution is defined for some values of eta < 0, this function does not reliably compute CDFs in this region.

    """

    beta = (eta + 1.5)/r 
    var_prior = scale * scipy.special.gamma(beta + 1/r)/scipy.special.gamma(beta)
    cheby = np.sqrt(var_prior/(tail_bound))
    n_tail = int(n_samples*tail_percent)
    
    # introduced additional bound in case chebyshev is unwieldy
    x_max = min(99, cheby) 
    if cheby < 120:
        n_tail = 0
        if debug:
            print(f"No tail")
    if debug:
        print(f"Chebyshev bound: {cheby}")

    xs = np.linspace(-x_max, x_max, n_samples-2*n_tail)
    xs = np.append(-np.logspace(np.log10(cheby), 2, n_tail), xs)
    xs = np.append(xs, np.logspace(2, np.log10(cheby), n_tail))

    prior_pdf = np.full(xs.shape, np.nan)

    for j, x in enumerate(xs):

        # Note that theta = variance, np.sqrt(theta) = SD
        def gauss_density(theta):
            return (1./(np.sqrt(2*np.pi)*np.sqrt(theta))) * np.exp(-0.5*(x**2/theta))

        def gen_gamma_density(theta):
            return (np.abs(r)/scipy.special.gamma(beta)) * (1/scale) * (theta/scale)**(r*beta - 1) * np.exp(-(theta/scale)**r)

        def integrand(theta):
            return gauss_density(theta) * gen_gamma_density(theta)

        if scipy_int:
            prior_pdf[j] = integrate.quad(integrand, 0, np.inf)[0]
        else:
            prior_pdf[j] = eng.compute_prior(float(r), float(eta), float(x), nargout=1)
    
    return xs, prior_pdf

In [ ]:

def gauss_density(theta):
            return (1./(np.sqrt(2*np.pi)*np.sqrt(theta))) * np.exp(-0.5*(x**2/theta))

def gen_gamma_density(theta):
    return (np.abs(r)/scipy.special.gamma(beta)) * (1/scale) * (theta/scale)**(r*beta - 1) * np.exp(-(theta/scale)**r)

def integrand(theta):
    return gauss_density(theta) * gen_gamma_density(theta)

